In [4]:
# TDO COMBINE TRAIN HISTORY DATA TO ONE FILE PER DAY
# TODO CALC TRAVALED PASSENGERN FROM TO FOR EACH DAY FOR EVERY STATION TO ONE! NEXT STATION

In [75]:
cwd = os.getcwd()
dep_table_path = os.path.join(cwd, 'generated/departure_tables_raw')
dep_table_path

'/home/jovyan/work/generated/departure_tables_raw'

In [104]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
raw_departure_tables = []
departure_tables_dates = []
departure_tables_dates_ts = []
# LOAD ALL CSV FILES INTO ONE BIG CHUNK
files = [ f for f in os.listdir( dep_table_path)] #list comprehension version.

for f in files:
    if(Path(f).suffix == ".csv"):
        filename = str(Path(f).stem)
        if '_' in filename and len(filename.split('_')) == 4:
            # THE GOAL IS TO COMBINE ALL TRAIN DEPARTURES FROM THE SAME DAY
            # SO THE FILE TIMESTAMP IN ITS NAME CORRESPDS TO THE DATE OF RECORD = TRAIN DEPARTURES
            # BY ADDING THE NEW DATE TO THE PD DF ITS IS POSSIBLE TO SORT FOR DATES LATER ON
            timestamp = int(filename.split('_')[3])
            date = datetime.fromtimestamp(timestamp)
            # THIS WILL BE ADDED TO THE DF AS NEW COLUMN AFTER LOADING
            combined_day = int(str(date.day) + str(date.month) + str(date.year))
            departure_tables_dates.append(combined_day)
            departure_tables_dates_ts.append(timestamp)
            # LOAD FILE TO DF
            abs_path = os.path.join(dep_table_path,f)
            df = pd.read_csv(abs_path,sep=';',encoding="utf-8",dtype = str)
            
            # ADD NEW COLUMNS
            df['date'] = pd.Series([date for x in range(len(df.index))])
            df['combined_date'] = pd.Series([combined_day for x in range(len(df.index))])
            
            raw_departure_tables.append(df)


            
# CREATE DATASET FROM DATE VALUES
df_departure_tables_dates = pd.DataFrame(data={'recorded_days':departure_tables_dates,'timestamp':departure_tables_dates_ts})
            
# AFER LOADING ALL RAW FILE IN CREATE ONE GIANT DATASET
departures_combined = pd.concat(raw_departure_tables)
departures_combined.head(5)


,type,name,line,number,start_station,stops,max_capacity,date,combined_date
0,RE,RE 30,30,3084,8000028,"8000028,8004759,8004284,8000284,",602,2021-11-29 11:20:05,29112021
1,RE,RE 38,38,59306,8000028,"8000028,8000974,8004936,8002605,8005895,800026...",602,2021-11-29 11:20:05,29112021
2,RE,RE 32,32,3460,8000028,"8000028,8004759,8004284,8000284,",602,2021-11-29 11:20:05,29112021
3,RE,RE 30,30,3410,8000028,"8000028,8001348,8000328,8004759,8002794,8000284,",602,2021-11-29 11:20:05,29112021
4,RE,RE 30,30,3087,8000028,"8000028,8004126,8002924,",602,2021-11-29 11:20:05,29112021


In [105]:
# REMOVE DUPLICATE TRAINS TRAIN_ID AND START_STATION
departures_combined = departures_combined.drop_duplicates(['number','start_station','combined_date'],keep= 'last')
departures_combined


,type,name,line,number,start_station,stops,max_capacity,date,combined_date
9666,RE,RE 12,12,19228,8000096,"8000096,8000235,8000038,8000925,8006171,800327...",602,2021-11-29 11:20:05,29112021
9667,RB,RB 13,13,19467,8000096,"8000096,8005769,8000180,8005424,8006030,800484...",426,2021-11-29 11:20:05,29112021
9668,RB,RB 18,18,19317,8000096,"8000096,8005769,8001920,8000302,8006331,800451...",426,2021-11-29 11:20:05,29112021
9669,RB,RB 19,19,17527,8000096,"8000096,8005769,8000180,8006479,8000016,800468...",426,2021-11-29 11:20:05,29112021
9670,RE,RE 87,87,50187,8000096,"8000096,8001055,8000177,8000322,8000163,8000073,",602,2021-11-29 11:20:05,29112021
...,...,...,...,...,...,...,...,...,...
9907,ICE,ICE 1031,1,1031,8002553,"8002553,8000098,8000086,8000085,8000207,",720,2021-11-28 23:15:16,28112021
9908,RE,RE 18,18,50703,8002553,"8002553,8010310,",602,2021-11-28 23:15:16,28112021
9909,ICE,ICE 703,18,703,8002553,"8002553,8010310,8010334,8010404,8098160,801111...",720,2021-11-28 23:15:16,28112021
9910,ICE,ICE 71,20,71,8002553,"8002553,8000238,8000152,8000128,8003200,800010...",720,2021-11-28 23:15:16,28112021


In [106]:
# LETS TEST THE DATA
# GET ALL TRAIN DEPARTURES FROM Bayreuth Hbf [8000028] @ 29.11.2021
# RV + FV + DB ONLY
tmp = departures_combined.loc[departures_combined['start_station'] == str(8000028)]
tmp = tmp.loc[tmp['combined_date'] == departure_tables_dates[0]]
tmp = tmp.reset_index()
tmp.head(100)

,index,type,name,line,number,start_station,stops,max_capacity,date,combined_date
0,10,RE,RE 38,38,59310,8000028,"8000028,8005895,8000267,8006017,8003476,800128...",602,2021-11-29 12:41:28,29112021
1,11,RE,RE 32,32,3464,8000028,"8000028,8004759,8002794,8000284,",602,2021-11-29 12:41:28,29112021
2,12,RE,RE 30,30,3414,8000028,"8000028,8001348,8000328,8004759,8004284,8000284,",602,2021-11-29 12:41:28,29112021
3,0,RE,RE 30,30,3091,8000028,"8000028,8004126,8002924,",602,2021-11-29 16:47:02,29112021
4,1,RE,RE 32,32,3476,8000028,"8000028,8001348,8004759,8002794,8000284,",602,2021-11-29 16:47:02,29112021
5,2,RE,RE 32,32,59312,8000028,"8000028,8005895,8000267,8003476,8003813,800128...",602,2021-11-29 16:47:02,29112021
6,7,RE,RE 38,38,59308,8000028,"8000028,8005895,8000267,8006017,8003476,800128...",602,2021-11-29 11:40:48,29112021
7,8,RE,RE 32,32,3462,8000028,"8000028,8004759,8002794,8000284,",602,2021-11-29 11:40:48,29112021
8,9,RE,RE 30,30,3412,8000028,"8000028,8001348,8000328,8004759,8004284,8000284,",602,2021-11-29 11:40:48,29112021
9,10,RE,RE 30,30,3089,8000028,"8000028,8004126,8002924,",602,2021-11-29 11:40:48,29112021


In [107]:
# COUNT THE UNIQUE TRAINS
#THE TRAIN_ID IS UNIQUE FOR EACH TRAIN AND IDENTIFIES THE TRAIN OVER ITS COMPLETE COURSE
# ONE UNIQUE ID = ONE ZUGLAUF
# FROM START TO END-STATION
tmp = departures_combined.loc[departures_combined['combined_date'] == departure_tables_dates[0]]
tmp['number'].nunique()

7720

In [108]:
# SAVE RESULTS
departures_combined.to_csv("./generated/4_departure_table.csv",sep=';',encoding="utf-8",index=False)
df_departure_tables_dates.to_csv("./generated/4_departure_table_day_list.csv",sep=';',encoding="utf-8",index=False)